In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, r2_score

In [108]:
filename = 'Concrete_Data_Yeh_final.csv'
variables = ['cement', 'slag', 'flyash', 'water', 'superplasticizer', 'coarseaggregate', 'fineaggregate', 'age', 'csMPa']

class PreProcessing:
    "The aim of this class is to successfully replace all the empty values from"
    "the file given, and to split the columns into their own series / arrays."
    def __init__(self, file):
        self.data = pd.read_csv(file)

    def checkNaN (self):
        return self.data.isnull().sum()

    def FillNaN (self) -> pd.DataFrame:
        # Here we used the .mean method to replace the NaN values from the original dataset. 
        # (Please check this as we may use an alternative fillna method)
        for i in self.data.columns:
            self.data[i].fillna(self.data[i].mean(), inplace = True)
            print(self.data[i].mean())
            if i == 'age':
                self.data[i] = np.log(self.data[i])
        return self.data

    
    def set_age_log(self):
        math.log(self.data[age])

#test = PreProcessing(filename).checkNaN()
#print('Number of NaN values in each column from the original csv file: \n' ,test)

#concretedata = PreProcessing(filename).FillNaN()
#concretedata.head(10)

print(concretedata.min())
print(concretedata.max())

cement              102.00
slag                  0.00
flyash                0.00
water               121.80
superplasticizer      0.00
coarseaggregate     801.00
fineaggregate       594.00
age                   0.00
csMPa                 2.33
dtype: float64
cement               540.000000
slag                 359.400000
flyash               200.100000
water                247.000000
superplasticizer      32.200000
coarseaggregate     1145.000000
fineaggregate        992.600000
age                    5.899897
csMPa                 82.600000
dtype: float64


In [128]:
%matplotlib qt
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button
 
# Create subplot
fig, ax = plt.subplots()
plt.subplots_adjust(bottom=0.5)
 
# Create and plot sine wave
t = np.arange(0.0, 100.0, 1)
s = np.exp(1-1/t)
l = plt.plot(t, s)
 
# Create axes for frequency and amplitude sliders
axcement = plt.axes([0.25, 0.15, 0.65, 0.03])
axslag = plt.axes([0.25, 0.1, 0.65, 0.03])
axflyash = plt.axes([0.25, 0.05, 0.65, 0.03])
axwater = plt.axes([0.25, 0.2, 0.65, 0.03])
axplast = plt.axes([0.25, 0.25, 0.65, 0.03])
axcoarse = plt.axes([0.25, 0.3, 0.65, 0.03])
#axfine = plt.axes([0.25, 0.35, 0.65, 0.03])

 
# Create a slider from 0.0 to 20.0 in axes axfreq
# with 3 as initial value
cement = Slider(axcement, 'Cement', 0.0, 20.0, 3)
Slag = Slider(axslag, 'Blast Furnace Slag', 0.0, 20.0, 3)
Fly_ash = Slider(axflyash, 'Fly Ash', 0.0, 20.0, 3)
Water = Slider(axwater, 'Water', 0.0, 20.0, 3)
Superplasticizer = Slider(axplast, 'Superplasticizer', 0.0, 20.0, 3)
Coarse = Slider(axcoarse, 'Coarse Aggregate', 0.0, 20.0, 3)
#Fine = Slider(axfine, 'Fine Aggregate', 0.0, 20.0, 3)

# Create function to be called when slider value is changed
 
def update(val):
    c = cement.val
    s = Slag.val
    f = Fly_ash.val
    w = Water.val
    su = Superplasticizer.val
    co = Coarse.val
    fi = Fine.val

    l.set_ydata(a*np.sin(2*np.pi*f*t))
# Call update function when slider value is changed
cement.on_changed(update)
Slag.on_changed(update)
 
plt.xlim([0, 50])
plt.ylim([0, 50])
ax.set(xlabel='Age (days)', ylabel='Compression Strength (MPa)')

# display graph
plt.show()

/var/folders/zb/x9zp6jsn0jd3nx3jwb6kx4vr0000gn/T/ipykernel_83700/1860904006.py:13: RuntimeWarning: divide by zero encountered in divide
  s = np.exp(1-1/t)
/var/folders/zb/x9zp6jsn0jd3nx3jwb6kx4vr0000gn/T/ipykernel_83700/1860904006.py:28: MatplotlibDeprecationWarning: Passing the valinit parameter of __init__() positionally is deprecated since Matplotlib 3.7; the parameter will become keyword-only two minor releases later.
  cement = Slider(axcement, 'Cement', 0.0, 20.0, 3)
/var/folders/zb/x9zp6jsn0jd3nx3jwb6kx4vr0000gn/T/ipykernel_83700/1860904006.py:29: MatplotlibDeprecationWarning: Passing the valinit parameter of __init__() positionally is deprecated since Matplotlib 3.7; the parameter will become keyword-only two minor releases later.
  Slag = Slider(axslag, 'Blast Furnace Slag', 0.0, 20.0, 3)
/var/folders/zb/x9zp6jsn0jd3nx3jwb6kx4vr0000gn/T/ipykernel_83700/1860904006.py:30: MatplotlibDeprecationWarning: Passing the valinit parameter of __init__() positionally is deprecated since 

In [240]:
%matplotlib qt
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button
 
# Create subplot
fig, ax = plt.subplots()
plt.subplots_adjust(bottom=0.5)
 
# Create and plot basic strength against age plot (for now this is just an approximate equation)
t = np.arange(0.0, 365.0, 1)
s = np.exp(1-1/t)
l = plt.plot(t, s)
 
Sliders = []

# Create a slider from min value to max value in for each input variable with the mean as initial value
for i in range(len(variables[:-1])):
    #plot each slider in a given location, each moving up by 0.5 to get a good spread
    axvariable = plt.axes([0.25, 0.05*(i + 1), 0.65, 0.03])
    variable = Slider(axvariable, variables[i], concretedata.min()[i], concretedata.max()[i], concretedata.mean()[i])
    #Add each slider to Sliders list for access later
    Sliders.append(variable)


#Below this hasnt been finished  
def update(val):
    for i in range(len(variables[:-2])):
        x = Sliders[i].val
        l.setydata(x*np.exp(1-1/t))
        print(x)

Sliders[i].on_changed(update)



# Call update function when slider value is changed
cement.on_changed(update)
Slag.on_changed(update)
 
plt.xlim([0, 50])
plt.ylim([0, 50])
ax.set(xlabel='Age (days)', ylabel='Compression Strength (MPa)')

# display graph
plt.show()

/var/folders/zb/x9zp6jsn0jd3nx3jwb6kx4vr0000gn/T/ipykernel_83700/664772796.py:13: RuntimeWarning: divide by zero encountered in divide
  s = np.exp(1-1/t)
/var/folders/zb/x9zp6jsn0jd3nx3jwb6kx4vr0000gn/T/ipykernel_83700/664772796.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  variable = Slider(axvariable, variables[i], concretedata.min()[i], concretedata.max()[i], concretedata.mean()[i])
/var/folders/zb/x9zp6jsn0jd3nx3jwb6kx4vr0000gn/T/ipykernel_83700/664772796.py:22: MatplotlibDeprecationWarning: Passing the valinit parameter of __init__() positionally is deprecated since Matplotlib 3.7; the parameter will become keyword-only two minor releases later.
  variable = Slider(axvariable, variables[i], concretedata.min()[i], concretedata.max()[i], concretedata.mean()[i])


In [124]:
# Import libraries
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button


# Create subplot
fig, ax = plt.subplots()
plt.subplots_adjust(bottom=0.35)
 
# Create and plot sine wave
t = np.arange(0.0, 1.0, 0.001)
s = 5 * np.sin(2 * np.pi * 3 * t)
l, = plt.plot(t, s)
 
# Create axes for frequency and amplitude sliders
axfreq = plt.axes([0.25, 0.15, 0.65, 0.03])
axamplitude = plt.axes([0.25, 0.1, 0.65, 0.03])
 
# Create a slider from 0.0 to 20.0 in axes axfreq
# with 3 as initial value
freq = Slider(axfreq, 'Frequency', 0.0, 20.0, 3)
 
# Create a slider from 0.0 to 10.0 in axes axfreq
# with 5 as initial value and valsteps of 1.0
amplitude = Slider(axamplitude, 'Amplitude', 0.0, 10.0, 5, valstep=1.0)

# Create function to be called when slider value is changed
 
def update(val):
    f = freq.val
    a = amplitude.val
    l.set_ydata(a*np.sin(2*np.pi*f*t))
 
# Call update function when slider value is changed
freq.on_changed(update)
amplitude.on_changed(update)
 
# display graph
plt.show()

/var/folders/zb/x9zp6jsn0jd3nx3jwb6kx4vr0000gn/T/ipykernel_83700/483828432.py:22: MatplotlibDeprecationWarning: Passing the valinit parameter of __init__() positionally is deprecated since Matplotlib 3.7; the parameter will become keyword-only two minor releases later.
  freq = Slider(axfreq, 'Frequency', 0.0, 20.0, 3)
/var/folders/zb/x9zp6jsn0jd3nx3jwb6kx4vr0000gn/T/ipykernel_83700/483828432.py:26: MatplotlibDeprecationWarning: Passing the valinit parameter of __init__() positionally is deprecated since Matplotlib 3.7; the parameter will become keyword-only two minor releases later.
  amplitude = Slider(axamplitude, 'Amplitude', 0.0, 10.0, 5, valstep=1.0)


In [265]:
import matplotlib.pyplot as plt
from matplotlib.widgets import Button, Slider

%matplotlib qt
variables = ['cement', 'slag', 'flyash', 'water', 'superplasticizer', 'coarseaggregate', 'fineaggregate', 'age', 'csMPa']
variable_coeff = [0.5, 0.78, 0.93, 0.56, 0.47, 0.76, 0.24]

def plot():
    #Define the regression function with all variables (for now this is just an approximate equation)
    def f(cement, slag, flyash, water, superplasticizer, coarseaggregate, fineaggregate, age):
        return np.exp(1-28/age)*(cement+slag+flyash+water+superplasticizer+coarseaggregate+fineaggregate)/(concretedata.mean().iloc[0]+concretedata.mean().iloc[1]+concretedata.mean().iloc[2]+concretedata.mean().iloc[3]+concretedata.mean().iloc[4]+concretedata.mean().iloc[5]+concretedata.mean().iloc[6])
    
    # Create subplot
    fig, ax = plt.subplots()
    plt.subplots_adjust(bottom=0.5) 
    ax.set(xlabel='Age (days)', ylabel='Compression Strength (MPa)')
    plt.ylim([0, 10])

    # Create and plot basic strength against age plot using the mean values of each variable for the initial plot
    t = np.arange(0.1, 365.0, 1)
    line = f(concretedata.mean().iloc[0], concretedata.mean().iloc[1], concretedata.mean().iloc[2], concretedata.mean().iloc[3], concretedata.mean().iloc[4], concretedata.mean().iloc[5], concretedata.mean().iloc[6], t)
    l, = plt.plot(t, line)

    #Create reset button to go back to the mean value
    resetax = fig.add_axes([0.025, 0.025, 0.1, 0.04])
    button = Button(resetax, 'Reset', color='0.5', hovercolor='0.975')
    
    #Create sliders for each variable and add them to list Sliders
    def create_axes():
        for i in range(len(variables[:-2])):
        #plot each slider in a given location, each moving up by 0.5 to get a good spread
            axvariable = plt.axes([0.25, 0.05*(i + 1), 0.65, 0.03])
            var_slider = Slider(axvariable, variables[i], concretedata.min().iloc[i], concretedata.max().iloc[i], concretedata.mean().iloc[i])
            #Add each slider to Sliders list for access later
            Sliders.append(var_slider)

            #This function checks for any change in the slider and will update the graph with the corresponding value
            def update(val):
                l.set_ydata(f(var_slider.val, var_slider.val, var_slider.val, var_slider.val, var_slider.val, var_slider.val, var_slider.val, t))
                fig.canvas.draw_idle()
            var_slider.on_changed(update)
            
            #This function implements the reset button such that the sliders are brought back to there initial value
            def reset(event):
                var_slider.reset()
            button.on_clicked(reset)
        '''
    I am currently getting the issue that the graph updating and 
    reset button is only working for the final iteration of the for 
    loop. i.e. fineaggregate. I am battling having to do each slider 
    indivually because that will be alot of lines
     '''

    create_axes()
    resetax._button = button
    plt.show()
w = plot()

/var/folders/zb/x9zp6jsn0jd3nx3jwb6kx4vr0000gn/T/ipykernel_83700/1363720935.py:33: MatplotlibDeprecationWarning: Passing the valinit parameter of __init__() positionally is deprecated since Matplotlib 3.7; the parameter will become keyword-only two minor releases later.
  var_slider = Slider(axvariable, variables[i], concretedata.min().iloc[i], concretedata.max().iloc[i], concretedata.mean().iloc[i])


In [251]:
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.widgets import Button, Slider


# The parametrized function to be plotted
def f(t, amplitude, frequency):
    return amplitude * np.sin(2 * np.pi * frequency * t)

t = np.linspace(0, 1, 1000)

# Define initial parameters
init_amplitude = 5
init_frequency = 3

# Create the figure and the line that we will manipulate
fig, ax = plt.subplots()
line, = ax.plot(t, f(t, init_amplitude, init_frequency), lw=2)
ax.set_xlabel('Time [s]')

# adjust the main plot to make room for the sliders
fig.subplots_adjust(left=0.25, bottom=0.25)

# Make a horizontal slider to control the frequency.
axfreq = fig.add_axes([0.25, 0.1, 0.65, 0.03])
freq_slider = Slider(
    ax=axfreq,
    label='Frequency [Hz]',
    valmin=0.1,
    valmax=30,
    valinit=init_frequency,
)

# Make a vertically oriented slider to control the amplitude
axamp = fig.add_axes([0.1, 0.25, 0.0225, 0.63])
amp_slider = Slider(
    ax=axamp,
    label="Amplitude",
    valmin=0,
    valmax=10,
    valinit=init_amplitude,
    orientation="vertical"
)


# The function to be called anytime a slider's value changes
def update(val):
    line.set_ydata(f(t, amp_slider.val, freq_slider.val))
    fig.canvas.draw_idle()


# register the update function with each slider
freq_slider.on_changed(update)
amp_slider.on_changed(update)

# Create a `matplotlib.widgets.Button` to reset the sliders to initial values.
resetax = fig.add_axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', hovercolor='0.975')


def reset(event):
    freq_slider.reset()
    amp_slider.reset()
button.on_clicked(reset)

plt.show()